In [28]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import itertools
import pandas as pd
import time
from multiprocessing import Pool, TimeoutError

In [29]:
N = 1200
CURRENT_MULTIPLIER= 1000.0
PARAM_GRANULARITY = 6
I_load = 0.01047

T = list(range(N))
T[0] = 0.001


In [30]:
def current_sigmoidal_2(t, A, B, C, D, I_neg, I_sd):
  I_measured = I_neg*(1.0-D*np.exp(-1.0*C/t)) -2*(I_neg*(1.0-D))*(0.5-1.0/(1.0+np.exp((t/A)**B))) - I_sd
  return I_measured

In [31]:
def current_sigmoidal_3(t, A, B, C, D, I_neg, I_sd):
  I_measured = (I_neg-I_sd)*(1.0-D*np.exp(-1.0*C/t)) -2*((I_neg+I_sd)*(1.0-D))*(0.5-1.0/(1.0+np.exp((t/A)**B))) - I_sd
  return I_measured  

In [32]:
def generate_current_data(params):
    #print(*params)
    return current_sigmoidal_2(T, *params), list(params)

In [33]:
def generate_all_data():
    training_data_input = []
    training_data_output = []
    A = np.linspace(80.0, 300.0, PARAM_GRANULARITY)
    B = np.linspace(0.1, 1.0, PARAM_GRANULARITY) 
    C = np.linspace(0.01, 1.0, PARAM_GRANULARITY)
    D = np.linspace(0.1, 1.0, PARAM_GRANULARITY)
    I_neg = np.linspace(-80.0, -10.0, PARAM_GRANULARITY)
    I_sd = np.linspace(-1.0, -0.01, PARAM_GRANULARITY)

    param_data = itertools.product(A, B, C, D, I_neg, I_sd) 
    #   for params in param_data:
    #     training_data_input.append(generate_current_data(params))
    #     training_data_output.append(params)

    # generate_current_data_vector = np.vectorize(generate_current_data)
    # training_data_input = generate_current_data_vector(param_data)

    all_data = list(map(generate_current_data, param_data))
    training_data_input = [x[0] for x in all_data]
    training_data_output = [x[1] for x in all_data]

    # p = Pool(4)
    # with p:
    #     training_data_input = list(p.map(generate_current_data, param_data))
    #     training_data_output = list(param_data)

    return training_data_input, training_data_output

In [34]:
start_time = time.time()
input, output = generate_all_data()
end_time = time.time()
print(end_time-start_time)

16.573036432266235


In [35]:
input_df = pd.DataFrame(input)

In [36]:
output_df = pd.DataFrame(output)

In [37]:
output_df.to_csv('/content/drive/MyDrive/Gdrive/startup/Novus Sentry/src/output.csv', header=False) 

In [38]:
input_df.to_csv('/content/drive/MyDrive/Gdrive/startup/Novus Sentry/src/input.csv', header=False)